In [44]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

df = pd.read_csv('Data/2017UTASP20180628.csv',encoding='cp932')

question_crosswalk = {'Q4_1':'防衛力強化',
                      'Q4_2':'先制攻撃',
                      'Q4_3':'北朝鮮圧力',
                      'Q4_4':'非核三原則',
                      'Q4_5':'靖国参拝',
                      'Q4_6':'小さな政府',
                      'Q4_7':'公共事業',
                      'Q4_8':'財政出動',
                      'Q4_9':'教育無償化',
                      'Q4_10':'累進課税',
                      'Q4_11':'個人権利の制約',
                      'Q4_12':'外人労働者',
                      'Q4_13':'原発再稼働',
                      'Q4_14':'夫婦別姓',
                      'Q4_15':'同性婚',
                      'Q4_16':'被選挙権引き下げ',
                      'Q4_17':'国会一院化'}
df = df.rename(columns=question_crosswalk)
questions = [value for value in question_crosswalk.values()]

df.head()

,ID,NAME,RESPONSE,PREFEC,DISTRICT,PR,PRBLOCK,PARTY,MINSHIN,INCUMB,...,Q9_6,Q9_7,Q9_8,Q9_9,Q1_1_FA,Q1_2_FA,Q1_3_FA,SQ7_1_1_FA,SQ7_1_2_FA,SQ7_1_3_FA
0,1,船橋＝利実,1,1,1,0,51,1,66,2,...,1,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN
1,2,道下＝大樹,1,1,1,0,51,8,2,1,...,5,5,5,5,経済・雇用,NaN,NaN,NaN,NaN,NaN
2,3,吉川＝貴盛,1,1,2,0,51,1,66,3,...,1,1,2,2,NaN,NaN,NaN,NaN,教育の充実,NaN
3,4,金倉＝昌俊,1,1,2,0,51,3,66,1,...,5,5,5,5,NaN,NaN,消費税10%中止,NaN,NaN,NaN
4,5,小和田＝康文,1,1,2,0,51,4,66,1,...,3,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df[questions] = KNNImputer().fit_transform(df[questions].replace(99,np.nan))
n_components = 3
pca = PCA(n_components=n_components).fit_transform(df[questions])
for index in range(n_components):
    df[f'第{index+1}軸'] = pca[:,index]

df[[f'第{index+1}軸' for index in range(3)]].corr()

,第1軸,第2軸,第3軸
第1軸,1.000000e+00,-1.556323e-16,4.830751e-17
第2軸,-1.556323e-16,1.000000e+00,-7.641375e-17
第3軸,4.830751e-17,-7.641375e-17,1.000000e+00


In [70]:
cands = df.loc[df['NAME'].isin(['河野＝太郎','菅＝義偉','石破＝茂','岸田＝文雄','安倍＝晋三','小泉＝進次郎']),['NAME'] + [f'第{index+1}軸' for index in range(3)]].set_index('NAME')

df['party'] = df['PARTY'].replace(range(1,12),['自民党','公明党','共産党','日本維新の会','社民党','その他','希望の党','立憲民主党','その他','その他','その他'])
party_means = df.groupby('party')[[f'第{index+1}軸' for index in range(3)]].mean()

data = pd.concat([cands,party_means])
data.to_excel('post_abe.xlsx')